# Warsztaty badawcze
## Praca domowa  1
### Paweł Morgen

In [7]:
import numpy as np
import pandas as pd
import dalex as dx
import pickle

In [39]:
housing = pd.read_csv('housing_preprocessed.csv', usecols = list(range(1,17)))
random_forest = pickle.load(open("random_forest", 'rb'))

In [40]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN,rooms_per_household,rooms_per_person
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0.0,0.0,0.0,1.0,0.0,6.984127,2.732919
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0.0,0.0,0.0,1.0,0.0,6.238137,2.956685
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0.0,0.0,0.0,1.0,0.0,8.288136,2.957661
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0.0,0.0,0.0,1.0,0.0,5.817352,2.283154
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0.0,0.0,0.0,1.0,0.0,6.281853,2.879646


In [41]:
predictors = housing.drop('median_house_value', axis = 1)
target = housing.loc[:, ['median_house_value']]
observation = predictors.loc[[420], :]

In [42]:
random_forest.predict(observation)

array([390127.7688711])

In [43]:
explainer = dx.Explainer(random_forest, predictors, target)

Preparation of a new explainer is initiated

  -> data              : 19608 rows 15 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 19608 values
  -> model_class       : sklearn.ensemble._forest.RandomForestRegressor (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_default at 0x0000029434DE4E50> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 5.77e+04, mean = 1.91e+05, max = 4.53e+05
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -2.17e+05, mean = 1.82e+02, max = 3.35e+05
  -> model_info        : package sklearn

A new explainer has been created!


In [44]:
pp = explainer.predict_parts(observation)

In [45]:
pp.plot()

In [46]:
observation2 = predictors.loc[1024]
observation3 = predictors.loc[34]
observation4 = predictors.loc[2021]
observation5 = predictors.loc[2137]

In [47]:
explainer.predict_parts(observation2).plot()

W `observation1` `rooms_per_household` na poziomie 6.67 ma wpływ dodatni, a w `observation2` - ujemny. By wyjaśnić ten paradoks, trzeba spojrzeć na szerszy kontekst.

`observation1` dotyczy posiadłości nad zatoką posiadanej przez dobrze zarabiające osoby. Możemy zakładać, że tu więcej przestrzeni to wyższa wartość.

`observation2` z kolei dotyczy nieruchomości w głębi lądu posiadanej przez słabiej zarabiające osoby.

In [49]:
explainer.predict_parts(observation4).plot()

W `observation1` dominującą zmienną było `median_income`, natomiast w `observation4` - `INLAND` oraz `latitude`.

Gdzie jest położone domostwo z `observation4` ?
![](california_latlong.jpg)

Są to przedmieścia Los Angeles, niedaleko Santa Barbary. Niewykluczone, że tam po prostu nie spotyka się posiadłości osób zamożniejszech.